# Method-I

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
plt.ioff() # interactive mode off
# from IPython.display import Latex
# from matplotlib.patches import Rectangle
from matplotlib import animation
import pandas as pd
# plt.rcParams['figure.figsize']=(16,9)
plt.rcParams.update({'font.size': 13})
plt.rcParams['animation.html'] = 'html5'
# plt.rcParams['text.usetex'] = True
# %config InlineBackend.close_figures=False # keep figures open in pyplot

In [2]:
def postproc(x1, x2, x3, T=10, N=100, P=[1,1,1], E=2000000, nu=0.2, limit=[-12,12,-12,12], animx=1):
    """
    X1,X2,X3 - material coordinates
    x1,x2,x3 - spacial coordinates
    t - current time(sec)
    T - total time(sec)
    N - no of time steps
    P - point at which the F,C,b,E,e,sigma,S are evaluated
    """
    ## Shape Functions and Their derivatives w.r.to X1,X2,X3 -- Linear Hexahedral Element
    # ---------------------------------------------------------- Shape Functions
    def N1(X1,X2,X3): return 0.125*(1-X1)*(1+X2)*(1+X3)
    def N2(X1,X2,X3): return 0.125*(1+X1)*(1+X2)*(1+X3)
    def N3(X1,X2,X3): return 0.125*(1-X1)*(1-X2)*(1+X3)
    def N4(X1,X2,X3): return 0.125*(1+X1)*(1-X2)*(1+X3)
    def N5(X1,X2,X3): return 0.125*(1-X1)*(1+X2)*(1-X3)
    def N6(X1,X2,X3): return 0.125*(1+X1)*(1+X2)*(1-X3)
    def N7(X1,X2,X3): return 0.125*(1-X1)*(1-X2)*(1-X3)
    def N8(X1,X2,X3): return 0.125*(1+X1)*(1-X2)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X1'
    def N1X1(X2,X3): return -0.125*(1+X2)*(1+X3)
    def N2X1(X2,X3): return  0.125*(1+X2)*(1+X3)
    def N3X1(X2,X3): return -0.125*(1-X2)*(1+X3)
    def N4X1(X2,X3): return  0.125*(1-X2)*(1+X3)
    def N5X1(X2,X3): return -0.125*(1+X2)*(1-X3)
    def N6X1(X2,X3): return  0.125*(1+X2)*(1-X3)
    def N7X1(X2,X3): return -0.125*(1-X2)*(1-X3)
    def N8X1(X2,X3): return  0.125*(1-X2)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X2'
    def N1X2(X1,X3): return  0.125*(1-X1)*(1+X3)
    def N2X2(X1,X3): return  0.125*(1+X1)*(1+X3)
    def N3X2(X1,X3): return -0.125*(1-X1)*(1+X3)
    def N4X2(X1,X3): return -0.125*(1+X1)*(1+X3)
    def N5X2(X1,X3): return  0.125*(1-X1)*(1-X3)
    def N6X2(X1,X3): return  0.125*(1+X1)*(1-X3)
    def N7X2(X1,X3): return -0.125*(1-X1)*(1-X3)
    def N8X2(X1,X3): return -0.125*(1+X1)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X3'
    def N1X3(X1,X2): return  0.125*(1-X1)*(1+X2)
    def N2X3(X1,X2): return  0.125*(1+X1)*(1+X2)
    def N3X3(X1,X2): return  0.125*(1-X1)*(1-X2)
    def N4X3(X1,X2): return  0.125*(1+X1)*(1-X2)
    def N5X3(X1,X2): return -0.125*(1-X1)*(1+X2)
    def N6X3(X1,X2): return -0.125*(1+X1)*(1+X2)
    def N7X3(X1,X2): return -0.125*(1-X1)*(1-X2)
    def N8X3(X1,X2): return -0.125*(1+X1)*(1-X2)

    def eqstress(S):
        Sd = S - (S.trace()/3)*I
        Seq = math.sqrt(1.5*np.tensordot(Sd,Sd))
        return Seq
    
    def eqstrain(E,nu):
        dummy = ((E[0,0]-E[1,1])**2 + E[1,1]-E[2,2])**2 + (E[2,2]-E[0,0])**2 + 3*(E[0,1]**2+E[1,2]**2+E[2,0]**2)
        Eeq = (1/(math.sqrt(2)*(1+nu)))*math.sqrt(dummy)
        return Eeq

    dt = T/N
    lmda = (E*nu)/((1+nu)*(1-2*nu))
    mu   = 1/(2*(1+nu))
    I = np.array([[1,0,0],[0,1,0],[0,0,1]])

    nodes_t0 = np.array([[-1,1,1],[1,1,1],[-1,-1,1],[1,-1,1],[-1,1,-1],[1,1,-1],[-1,-1,-1],[1,-1,-1]])
    xnodes_ndt = np.zeros((N+1,8,3))
    xnodes_ndt[0,:,:] = nodes_t0
    unodes_ndt = np.zeros((N+1,8,3))
    
    point_ndt = np.zeros((N+1,3))
    point_ndt[0,:] = P
    grad_u_P = np.zeros((N+1,3,3))
    F_P = np.zeros((N+1,3,3))
    C_P = np.zeros((N+1,3,3))
    b_P = np.zeros((N+1,3,3))
    E_P = np.zeros((N+1,3,3))
    e_P = np.zeros((N+1,3,3))
    S_P = np.zeros((N+1,3,3))
    sigma_P = np.zeros((N+1,3,3))
    Eeq_P = np.zeros((N+1))
    eeq_P = np.zeros((N+1))
    Seq_P = np.zeros((N+1))
    sigmaeq_P = np.zeros((N+1))


    for n in range(N):
        t = n*dt
        # Calculation of nodal positions and displacements
        for i in range(8):
            X1 = xnodes_ndt[0,i,0]
            X2 = xnodes_ndt[0,i,1]
            X3 = xnodes_ndt[0,i,2]
            xnodes_ndt[n+1,i,0] = eval(x1)
            xnodes_ndt[n+1,i,1] = eval(x2)
            xnodes_ndt[n+1,i,2] = eval(x3)
            unodes_ndt[n+1,i,0] = xnodes_ndt[n+1,i,0] - X1
            unodes_ndt[n+1,i,1] = xnodes_ndt[n+1,i,1] - X2
            unodes_ndt[n+1,i,2] = xnodes_ndt[n+1,i,2] - X3
            
        # Calculation of required quantities at point 'P'
        X1 = P[0]
        X2 = P[1]
        X3 = P[2]
        point_ndt[n+1,0] = eval(x1)
        point_ndt[n+1,1] = eval(x2)
        point_ndt[n+1,2] = eval(x3)
        NX1 = np.array([N1X1(X2,X3), N2X1(X2,X3), N3X1(X2,X3), N4X1(X2,X3), N5X1(X2,X3), N6X1(X2,X3), N7X1(X2,X3), N8X1(X2,X3)])
        NX2 = np.array([N1X2(X1,X3), N2X2(X1,X3), N3X2(X1,X3), N4X2(X1,X3), N5X2(X1,X3), N6X2(X1,X3), N7X2(X1,X3), N8X2(X1,X3)])
        NX3 = np.array([N1X3(X1,X2), N2X3(X1,X2), N3X3(X1,X2), N4X3(X1,X2), N5X3(X1,X2), N6X3(X1,X2), N7X3(X1,X2), N8X3(X1,X2)])
        NX  = np.array([NX1, NX2, NX3])
        for i in range(3):
            for j in range(3):
                grad_u_P[n+1,i,j] = np.dot(unodes_ndt[n+1,:,i],NX[j])
        F_P[n+1,:,:] = I + grad_u_P[n+1,:,:]
        C_P[n+1,:,:] = np.dot(F_P[n+1,:,:].T,F_P[n+1,:,:])
        b_P[n+1,:,:] = np.dot(F_P[n+1,:,:],F_P[n+1,:,:].T)
        E_P[n+1,:,:] = 0.5*(C_P[n+1,:,:] - I)
        e_P[n+1,:,:] = 0.5*(I - np.linalg.inv(b_P[n+1,:,:]))
        J = math.sqrt(np.linalg.det(C_P[n+1,:,:]))
        S_P[n+1,:,:] = mu*(I-np.linalg.inv(C_P[n+1,:,:]))+lmda*math.log(J)*np.linalg.inv(C_P[n+1,:,:]) # Neo-Hookean Material
        # sigma_P[n+1,:,:] = (1/J)*mu*(b_P[n+1,:,:]-I)+lmda*math.log(J)*I
        sigma_P[n+1,:,:] = (np.dot(F_P[n+1,:,:],np.dot(S_P[n+1,:,:],F_P[n+1,:,:].T)))/(J)
        Eeq_P[n+1] = eqstrain(E_P[n+1,:,:],nu)
        eeq_P[n+1] = eqstrain(e_P[n+1,:,:],nu)
        Seq_P[n+1] = eqstress(S_P[n+1,:,:])
        sigmaeq_P[n+1] = eqstress(sigma_P[n+1,:,:])

    # DataFrame
    dN = round(N/round(T))
    output = pd.DataFrame({'Time[sec]':range(round(T)+1),'Eeq': Eeq_P[0:N+1:dN], 'Seq': Seq_P[0:N+1:dN], 'eeq': eeq_P[0:N+1:dN], 'sigmaeq': sigmaeq_P[0:N+1:dN]}).set_index('Time[sec]')
    display(output)
    
    # create a figure and axes
    fig = plt.figure(figsize=(16,8), dpi=100)
    dmap = plt.subplot(1,2,1)   
    SvE = plt.subplot(1,2,2)
    sve = SvE.twinx().twiny()

    # set up the subplots as needed
    dmap.set_xlim(limit[0], limit[1])            
    dmap.set_ylim(limit[2], limit[3])
    dmap.axvline(c="black",lw=1,ls='--')
    dmap.axhline(c="black",lw=1,ls='--')
    dmap.set_aspect('equal')
    dmap.set_xlabel('x1')
    dmap.set_ylabel('x2')
    # dmap.set_title('Deformation Map')

    # Second-Piola Kirchhoff's stress vs Lagrange strain
    lxlim = round(Eeq_P.min(),2)
    rxlim = round(Eeq_P.max(),2)
    lylim = round(Seq_P.min(),2)
    rylim = round(Seq_P.max(),2)
    if lxlim!=0 or rxlim!=0:
        if lxlim == 0:
            lxlim = -(rxlim/10)
        SvE.set_xlim(lxlim,rxlim)    
    if lylim!=0 or rylim!=0:
        if lylim == 0:
            lylim = -(rylim/10)
        SvE.set_ylim(lylim,rylim)
    SvE.axvline(c="black",lw=1,ls='--')
    SvE.axhline(c="black",lw=1,ls='--')
    SvE.set_box_aspect(1)
    # SvE.set_adjustable('datalim')
    # SvE.axis('tight')
    # SvE.set_xlabel(r'$||e||$ or $||E||$')
    # SvE.set_ylabel(r'$||\sigma||$ or $||S||$')
    SvE.set_xlabel('Strain')
    SvE.set_ylabel('Stress')
    # SvE.set_xlabel(r'$\|\underline{\underline{e}}\|$ / $\|underline{\underline{E}}\|$', fontsize=16)
    # SvE.set_ylabel(r'$\|\underline{\underline{\sigma}}\|$ / $\|\underline{\underline{S}}\|$', fontsize=16)
    SvE.set_title("Stress vs Strain at 'P'", y=1, pad=35)

    # Cauchy stress vs Euler strain
    lxlim = round(eeq_P.min(),2)
    rxlim = round(eeq_P.max(),2)
    lylim = round(sigmaeq_P.min(),2)
    rylim = round(sigmaeq_P.max(),2)
    if lxlim!=0 or rxlim!=0:
        if lxlim == 0:
            lxlim = -(rxlim/10)
        sve.set_xlim(lxlim,rxlim)    
    if lylim!=0 or rylim!=0:
        if lylim == 0:
            lylim = -(rylim/10)
        sve.set_ylim(lylim,rylim)
    # sve.set_xlabel(r'$||e||$')
    # sve.set_ylabel(r'$||\sigma||$')
    

    # create objects that will change in the animation.
    # These are initially empty, and will be given new values for each frame in the animation.
    timestep = dmap.set_title('', y=1, pad=35)
    face5687, = dmap.plot([], [], c="grey", lw=1, ls='--')
    edge15, = dmap.plot([], [], c="grey", lw=1, ls='--')
    edge26, = dmap.plot([], [], c="grey", lw=1, ls='--')
    edge48, = dmap.plot([], [], c="grey", lw=1, ls='--')
    edge37, = dmap.plot([], [], c="grey", lw=1, ls='--')
    face1243, = dmap.plot([], [], 'tab:blue', lw=2.5)
    point, = dmap.plot([], [],'ob', ms=8, label="The point 'P'")
    dmap.legend(loc='upper right')

    SeqvEeq, = SvE.plot([], [], 'tab:blue', lw=2.5, label=r'$||S||$ vs $||E||$')
    SvEpoint, = SvE.plot([], [], 'tab:blue', marker='o', ms=8)
    SvElinex, = SvE.plot([], [], 'tab:blue', lw=1.5, ls='--')
    SvEliney, = SvE.plot([], [], 'tab:blue', lw=1.5, ls='--')
    SvE.legend(loc='upper left')
    sigmaVe, = sve.plot([], [], 'tab:orange', lw=2.5, label=r'$||\sigma||$ vs $||e||$')
    svepoint, = sve.plot([], [], 'tab:orange', marker='o', ms=8)
    svelinex, = sve.plot([], [], 'tab:orange', lw=1.5, ls='--')
    sveliney, = sve.plot([], [], 'tab:orange', lw=1.5, ls='--')
    sve.legend(loc='upper right')

    # animation function. This is called sequentially
    def drawframe(n, dt=dt, x1x2=xnodes_ndt, S=Seq_P, E=Eeq_P, sigma=sigmaeq_P, e=eeq_P, P=point_ndt):
        face5687.set_data(x1x2[n,(4,5,7,6,4),0], x1x2[n,(4,5,7,6,4),1])
        edge15.set_data(x1x2[n,(0,4),0], x1x2[n,(0,4),1])
        edge26.set_data(x1x2[n,(1,5),0], x1x2[n,(1,5),1])
        edge48.set_data(x1x2[n,(3,7),0], x1x2[n,(3,7),1])
        edge37.set_data(x1x2[n,(2,6),0], x1x2[n,(2,6),1])
        face1243.set_data(x1x2[n,(0,1,3,2,0),0], x1x2[n,(0,1,3,2,0),1])
        point.set_data(point_ndt[n,0],point_ndt[n,1])
        timestep.set_text('Deformed config. at t = {:04.1f} sec'.format(round(n*dt,1)))

        SeqvEeq.set_data(E[:n+1],S[:n+1])
        SvEpoint.set_data(E[n],S[n])
        SvElinex.set_data([E[n],-1e100],[S[n],S[n]])
        SvEliney.set_data([E[n],E[n]],[S[n],-1e100])
        sigmaVe.set_data(e[:n+1],sigma[:n+1])
        svepoint.set_data(e[n],sigma[n])
        svelinex.set_data([e[n],1e100],[sigma[n],sigma[n]])
        sveliney.set_data([e[n],e[n]],[sigma[n],1e100])
        return (face1243,face5687)
    
    anim = animation.FuncAnimation(fig, drawframe, frames=N+1, interval=20/animx, blit=True)
    return(anim)

## 1. Translation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
+
\begin{bmatrix}
    t \\ t \\ 0
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1+t\\
    x_2 &= X_2+t\\
    x_3 &= X_3
\end{align*}
$$

In [3]:
x1 = 'X1+t'
x2 = 'X2+t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=300, P = [1,1,1], limit=[-2, 12, -2, 12], animx=1)


,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0


## 2. Rotation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    \cos\omega t & -\sin\omega t & 0 \\
    \sin\omega t & \cos\omega t & 0 \\
    0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1\cos\omega t - X_2\sin\omega t \\
    x_2 &= X_1\sin\omega t + X_2\cos\omega t \\
    x_3 &= X_3
\end{align*}
$$

$$
\omega t = 90^o \implies \omega = 9^o
$$

In [4]:
x1 = 'X1*math.cos(math.radians(9*t))-X2*math.sin(math.radians(9*t))'
x2 = 'X1*math.sin(math.radians(9*t))+X2*math.cos(math.radians(9*t))'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=300, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.5)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,7.329344e-17,9.275740e-17,9.263807e-17,9.275740e-17
2,8.584826e-18,1.214078e-17,8.584826e-18,1.214078e-17
3,1.303929e-17,1.844034e-17,1.303929e-17,1.844034e-17
4,9.270122e-17,9.288349e-17,9.270122e-17,9.288349e-17
5,9.252121e-17,9.252384e-17,9.252121e-17,9.252384e-17
6,9.289617e-17,9.327222e-17,9.289617e-17,9.327222e-17
7,6.596152e-17,9.328367e-17,6.596152e-17,9.328367e-17
8,2.263451e-18,3.201003e-18,2.263451e-18,3.201003e-18


## 3. Translation + Rotation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    \cos\omega t & -\sin\omega t & 0 \\
    \sin\omega t & \cos\omega t & 0 \\
    0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
+
\begin{bmatrix}
    t \\ t \\ 0
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1\cos\omega t - X_2\sin\omega t + t\\
    x_2 &= X_1\sin\omega t + X_2\cos\omega t + t\\
    x_3 &= X_3
\end{align*}
$$
$$
\omega t = 90^o \implies \omega = 9^o
$$

In [5]:
x1 = 'X1*math.cos(math.radians(9*t))-X2*math.sin(math.radians(9*t))+t'
x2 = 'X1*math.sin(math.radians(9*t))+X2*math.cos(math.radians(9*t))+t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=300, P = [1,1,1], limit=[-2, 12, -2, 12], animx=1)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,3.290129e-17,9.265396e-17,6.551624e-17,9.265396e-17
2,8.584826e-18,1.214078e-17,8.584826e-18,1.214078e-17
3,1.001872e-17,1.416861e-17,1.001872e-17,1.416861e-17
4,2.895732e-16,4.095184e-16,2.895732e-16,4.095184e-16
5,6.542067e-17,9.251880e-17,6.542067e-17,9.251880e-17
6,9.289617e-17,9.327222e-17,9.289617e-17,9.327222e-17
7,1.850553e-16,1.850734e-16,1.850553e-16,1.850734e-16
8,4.757153e-16,5.304869e-16,4.757153e-16,5.304869e-16


## 4. Pure Shear
$$
\begin{align*}
    x_1 &= X_1 + X_2t\\
    x_2 &= X_2\\
    x_3 &= X_3
\end{align*}
$$

In [6]:
x1 = 'X1+X2*t'
x2 = 'X2'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=300, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000,0.000000,0.000000
1,0.637587,0.798927,0.514646,0.798927
2,3.490712,2.147473,1.462803,2.147473
3,14.085518,4.246381,8.946836,4.246381
4,41.156524,7.153768,31.899394,7.153768
5,96.941359,10.885313,82.411577,10.885313
6,197.223998,15.446157,176.248640,15.446157
7,361.328606,20.838314,332.730914,20.838315
8,612.116711,27.062692,574.718842,27.062691


## 5. Generic
$$
\begin{align*}
    x_1 &= X_1e^t + X_3(e^t-1)\\
    x_2 &= X_2 + X_3(e^t-e^{-t})\\
    x_3 &= X_3
\end{align*}
$$

In [7]:
x1 = 'X1*math.exp(t)+X3*(math.exp(t)-1)'
x2 = 'X2+X3*(math.exp(t)-math.exp(-t))'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=300, P = [1,1,1], limit=[-7, 7, -7, 7], animx=0.5)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
1,3.828502e+00,3.809928e+06,2.774895e+00,1.729104
2,3.447412e+02,5.657062e+07,2.137109e+01,6.290569
3,2.066255e+04,6.235008e+08,1.574193e+02,18.812097
4,1.143211e+06,6.147602e+09,1.162154e+03,52.878235
5,6.253190e+07,5.685977e+10,8.585999e+03,145.487637
6,3.414981e+09,5.047111e+11,6.344114e+04,397.228690
7,1.864580e+11,4.354343e+12,4.687688e+05,1081.532725
8,1.018031e+13,3.679280e+13,3.463758e+06,2941.664273


## 6. Biaxial Tension
$$
\begin{align*}
    x_1 &= X_1 + X_1t\\
    x_2 &= X_2 + X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [8]:
x1 = 'X1+X1*t'
x2 = 'X2+X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=300, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000e+00,0.000000,0.000000
1,1.194907,5.571939e+05,0.308939,0.308939
2,3.250463,1.070980e+06,0.369324,0.369324
3,6.139352,1.433724e+06,0.390185,0.390185
4,9.861574,1.708639e+06,0.399776,0.399776
5,14.417130,1.928906e+06,0.404963,0.404963
6,19.806019,2.112379e+06,0.408082,0.408082
7,26.028241,2.269520e+06,0.410102,0.410102
8,33.083796,2.406924e+06,0.411484,0.411484


## 7. Biaxial Compression
$$
\begin{align*}
    x_1 &= X_1 - X_1t\\
    x_2 &= X_2 - X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [9]:
x1 = 'X1-X1*t'
x2 = 'X2-X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=0.8, N=300, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.2)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000e+00,0.000000,0.000000
1,0.399553,4.140602e+07,9.727681,9.727681


## 8. Plane Strain Compression
$$
\begin{align*}
    x_1 &= X_1 + X_1t\\
    x_2 &= X_2 - X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [10]:
x1 = 'X1+X1*t'
x2 = 'X2-X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=0.8, N=300, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.2)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000e+00,0.000000,0.000000
1,1.382141,1.329643e+07,78.235515,3.242038
